<a href="https://colab.research.google.com/github/MONISHKA1607/Accelerated_Data_Science_UCS547/blob/main/Assignment2_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q1
Identify !, %, and %% used in cell in Google Colab.

In [4]:
%time x = sum(range(1000000))

CPU times: user 18.5 ms, sys: 0 ns, total: 18.5 ms
Wall time: 18.5 ms


In [5]:
%%writefile hello.cu
#include<stdio.h>
int main(){
    printf("Hello from CUDA file!\n");
    return 0;
}

Writing hello.cu


In [8]:
!nvcc -arch=sm_75 hello.cu -o hello
!./hello

Hello from CUDA file!


Q2 Identify all key nvidia-smi commands with multiple options

In [9]:
!nvidia-smi

Sun Feb 15 16:03:44 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
!nvidia-smi -q



==============NVSMI LOG==============

Timestamp                                 : Sun Feb 15 16:03:52 2026
Driver Version                            : 580.82.07
CUDA Version                              : 13.0

Attached GPUs                             : 1
GPU 00000000:00:04.0
    Product Name                          : Tesla T4
    Product Brand                         : NVIDIA
    Product Architecture                  : Turing
    Display Mode                          : Requested functionality has been deprecated
    Display Attached                      : Yes
    Display Active                        : Disabled
    Persistence Mode                      : Disabled
    Addressing Mode                       : None
    MIG Mode
        Current                           : N/A
        Pending                           : N/A
    Accounting Mode                       : Disabled
    Accounting Mode Buffer Size           : 4000
    Driver Model
        Current                           : N/

In [11]:
!nvidia-smi --query-gpu=name,memory.total,memory.used,memory.free --format=csv


name, memory.total [MiB], memory.used [MiB], memory.free [MiB]
Tesla T4, 15360 MiB, 0 MiB, 14913 MiB


In [12]:
!nvidia-smi -l 1


Sun Feb 15 16:04:15 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Q3 Debug common CUDA errors (zero output, incorrect indexing, PTX errors)

In [13]:
%%writefile zero_output.cu
#include<stdio.h>

__global__ void test(){
    printf("Hello from GPU\n");
}

int main(){
    test<<<1,1>>>();
    cudaDeviceSynchronize();
    return 0;
}

Writing zero_output.cu


In [16]:
!nvcc -arch=sm_75 zero_output.cu -o zero_output

In [17]:
!./zero_output

Hello from GPU


B) Incorrect Indexing Fix

In [18]:
%%writefile correct_index.cu
#include<stdio.h>

__global__ void kernel(){
    int id = blockIdx.x * blockDim.x + threadIdx.x;
    printf("Thread ID: %d\n", id);
}

int main(){
    kernel<<<2,4>>>();
    cudaDeviceSynchronize();
    return 0;
}


Writing correct_index.cu


In [20]:
!nvcc -arch=sm_75 correct_index.cu -o correct_index


In [21]:
!./correct_index

Thread ID: 0
Thread ID: 1
Thread ID: 2
Thread ID: 3
Thread ID: 4
Thread ID: 5
Thread ID: 6
Thread ID: 7


C) PTX Architecture Fix

In [22]:
!nvcc -arch=sm_75 correct_index.cu -o correct_index

Q4 Write a CUDA C/C++ program to demonstrate GPU kernel execution and thread indexing.
a. Launch a CUDA kernel using: 1 block and 8 threads
b. Each thread must print: Hello from GPU thread <global_thread_id>
c. Compute the global thread ID using: global_thread_id = blockIdx.x * blockDim.x +threadIdx.x
d. Clearly separate: Host code (CPU) & Device code (GPU kernel)

In [23]:
%%writefile thread_index.cu
#include<stdio.h>

// DEVICE CODE
__global__ void myKernel() {
    int global_thread_id = blockIdx.x * blockDim.x + threadIdx.x;
    printf("Hello from GPU thread %d\n", global_thread_id);
}

// HOST CODE
int main() {
    printf("Launching kernel...\n");

    myKernel<<<1,8>>>();

    cudaDeviceSynchronize();

    printf("Back to CPU\n");
    return 0;
}


Writing thread_index.cu


In [25]:
!nvcc -arch=sm_75 thread_index.cu -o thread_index

In [26]:
!./thread_index


Launching kernel...
Hello from GPU thread 0
Hello from GPU thread 1
Hello from GPU thread 2
Hello from GPU thread 3
Hello from GPU thread 4
Hello from GPU thread 5
Hello from GPU thread 6
Hello from GPU thread 7
Back to CPU


Q5 Write a CUDA program to demonstrate host and device memory separation.
a. Create an integer array of size 5 on the host (CPU).
b. Allocate corresponding memory on the device (GPU) using cudaMalloc().
c. Copy data from host to device using cudaMemcpy().
d. Launch a kernel where GPU threads print values from device memory.
e. Copy the data back from device to host and print it on CPU.

In [28]:
%%writefile memory_demo.cu
#include<stdio.h>
#include<cuda_runtime.h>

#define N 5
__global__ void printDevice(int *d_arr) {
    int id = threadIdx.x;
    printf("GPU Thread %d: Value = %d\n", id, d_arr[id]);
}


int main() {

    int h_arr[N] = {10, 20, 30, 40, 50};

    int *d_arr;
    int size = N * sizeof(int);

    cudaMalloc((void**)&d_arr, size);

    cudaMemcpy(d_arr, h_arr, size, cudaMemcpyHostToDevice);

    printDevice<<<1,N>>>(d_arr);
    cudaDeviceSynchronize();

    cudaMemcpy(h_arr, d_arr, size, cudaMemcpyDeviceToHost);

    printf("\nBack on CPU:\n");
    for(int i=0;i<N;i++)
        printf("%d ", h_arr[i]);

    printf("\n");

    cudaFree(d_arr);

    return 0;
}


Overwriting memory_demo.cu


In [30]:
!nvcc -arch=sm_75 memory_demo.cu -o memory_demo


In [31]:
!./memory_demo


GPU Thread 0: Value = 10
GPU Thread 1: Value = 20
GPU Thread 2: Value = 30
GPU Thread 3: Value = 40
GPU Thread 4: Value = 50

Back on CPU:
10 20 30 40 50 


Q6 Compare CPU times of List/tuple with Numpy arrays.

In [32]:
import time
import numpy as np

N = 10_000_000
a = list(range(N))
b = list(range(N))

start = time.time()
c = [a[i] + b[i] for i in range(N)]
end = time.time()
print("Python List Time:", end - start)

a_np = np.arange(N)
b_np = np.arange(N)

start = time.time()
c_np = a_np + b_np
end = time.time()
print("NumPy Time:", end - start)


Python List Time: 1.150904893875122
NumPy Time: 0.03157544136047363
